In [ ]:
import asyncio
import socketio
sio = socketio.AsyncClient()

file1 = "ru_lock_unlock_normal1_simple.log"
file2 = "ru_lock_unlock_dpd_hw_fault_simple.log"

await sio.connect('http://127.0.0.1:8000', namespaces=['/TextAnalysis/FileContainer', '/TextAnalysis/TextFileCompare'])
await sio.emit('new_file', [f'D:\\Projects\\ericsson_flow\\new_files\\{file1}'], namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(0.2)
await sio.emit('load_config', 'D:\\projects\\ericsson_flow\\new_files\\6419config.ecfg', namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(10)
await sio.emit('new_file', [f'D:\\Projects\\ericsson_flow\\new_files\\{file2}'], namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(0.2)
await sio.emit('load_config', 'D:\\projects\\ericsson_flow\\new_files\\6419config.ecfg', namespace='/TextAnalysis/FileContainer')
await asyncio.sleep(10)
await sio.emit('exec', {'first_file_namespace': f'/TextAnalysis/FileContainer/{file1}', 'second_file_namespace': f'/TextAnalysis/FileContainer/{file2}'}, namespace='/TextAnalysis/TextFileCompare')

In [ ]:
sio1 = socketio.AsyncClient()
await sio1.connect('http://127.0.0.1:8000', namespaces=[
'/TextAnalysis/FileContainer',
'/TextAnalysis/TextFileCompare',
'/TextAnalysis/FileContainer/ru_lock_unlock_normal1_simple.log/TextFileFunction/SearchFunction/txlProcBranch0',
'/TextAnalysis/FileContainer/ru_lock_unlock_dpd_hw_fault_simple.log/TextFileFunction/SearchFunction/txlProcBranch0'
])

In [ ]:
model1 = {}
model2 = {}

def set_data1(model):
    model1.update(model)
def set_data2(model):
    model2.update(model)
await sio1.emit('model', namespace='/TextAnalysis/FileContainer/ru_lock_unlock_normal1_simple.log/TextFileFunction/SearchFunction/txlProcBranch0', callback=set_data1)
await asyncio.sleep(0.2)
await sio1.emit('model', namespace='/TextAnalysis/FileContainer/ru_lock_unlock_dpd_hw_fault_simple.log/TextFileFunction/SearchFunction/txlProcBranch0', callback=set_data2)

In [ ]:
import re
string = '       air_mongoose com_ericsson_trithread INFO cpu_id  process txlProcBranch fileAndLine spiMaster cc  msg Warning Slow response to SPI_SEND_REQ for device paCtrlDevice   took ms '
re.sub(' '+"+", ' ', string)

In [ ]:
s = 'Power measurement'
for item in model1['res_clean_lines']:
    if s in item:
        print('model1', item)
        
for item in model2['res_clean_lines']:
    if s in item:
        print('model2', item)

In [ ]:
	[14:56:47.927884604] (+0.001409563) air6419_mongoose com_ericsson_trithread:INFO: { cpu_id = 2 }, { process = "WorkerTaskAas", fileAndLine = "algAasHelper.cc:659", msg = "Missing subbandMappingExt param, using default fullband instead." }

In [ ]:
model2['res_clean_lines']

In [ ]:
class A(object):
    def __init__(self):
        self.a = 1
        self.b = 2
        
    def __getattribute__(self, obj):
        print(obj)
        return object.__getattribute__(self, obj)
    
c = A()
c.a * c.b

In [ ]:
# from asyncio import get_event_loop
from text_analysis import *

# loop = get_event_loop()
# loop.run_until_complete(TextAnalysisModel('parallel'))

web.run_app(app, host="127.0.0.1", port=8000)

In [ ]:
from dateutil.parser import parse as dp

str(dp('221210-12:40:51', yearfirst=True))

In [ ]:
if '2022-12-11 01:12:21.105110' > '2022-12-11 01:12:22':
    print(True)

In [ ]:
import re
import pandas as pd
from parse import parse

str1 = '230228-16:19:06+0100 172.21.11.66 23.0b MSRBS_NODE_MODEL_22.Q2_566.28125.116_3317 stopfile=/tmp/11737'
str2 = 'BXP_7:     769,dl-1  disabled        false    essFdd150_Id2       B     dl    2162500         BandI      0,       0    469            469     SETUP     SETUP'
# exp_extract = '{}MSRBS_NODE_MODEL_{version}_{}'
exp_extract = '{bxp}:{:s}{carrierId}{:s}{Enabled}{:s}{RealRelease}{:s}{carrierType}{:s}{rfPort}{:s}dl{:s}{Frequency}{:s}{Band}{:s}{Arfcn_min}{:s}{Arfcn_max}{:s}{Power}{:s}{ReservedPower}{:s}{tr}{:s}{bdconf}'
r_extract = parse(exp_extract, str2)
print(r_extract.named)
# print(r_extract)

In [ ]:
lines = []
for index in self.text_file.data['test'].res_lines:
    lines.append(self.text_file.lines[index])

with open('D:\\projects\\ericsson_flow\\new_files\\ru_lock_unlock_normal2_simple.log', 'w', encoding='utf-8') as f:
    f.write(''.join(lines))
    
# jesd|dfe|radiosw|rProxyMedian.cc # (txlProcBranch0|TxBranchCtrl0).*event com_ericsson_trithread:INFO

In [ ]:
data = self.text_file.data['dl'].res_key_value
file = pd.DataFrame(pd.Series(data['Band']['value']), columns=['Band'])

file['rfPort'] = pd.Series(data['rfPort']['value'])
file['rfPort'] = 'Port_' + file['rfPort']
file['carrierType'] = pd.Series(data['carrierType']['value'])
file['Power'] = pd.Series(data['Power']['value'])

result = {
            'SiteName': self.text_file.file_name.split('_radio4480')[0],
            'Version': self.text_file.data['version'].res_key_value['version']['value'][0],
            'RadioQuantity': len(self.text_file.data['device'].res_lines)
         }
for band,port,carrier,power in file.values:
    result[band+'_'+port+'_carrier_config'] = carrier
    result[band+'_'+port+'_output_power_config'] = power

self.result = result

In [ ]:
import json
import pandas as pd

data = pd.read_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\KaiHong_4480config_origin_data.csv')
data = data.fillna('')
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
band1.extend(band3)
columns = band1
total = len(data)
res = data.groupby(columns).size().reset_index(name='RadioAmount')
for index,item in enumerate(json.loads(res.to_json(orient='records'))):
    band1_dbm = []
    band3_dbm = []
    total_dbm = 0
    for key in item.keys():
        if ('output_power_BandIII' in key) & (item[key] != ''):
            band3_dbm.append(item[key])
        elif ('output_power_BandI' in key) & (item[key] != ''):
            band1_dbm.append(item[key])
    band1_dbm = sum(band1_dbm) if len(band1_dbm) > 0 else 0
    band3_dbm = sum(band3_dbm) if len(band3_dbm) > 0 else 0
    total_dbm = band1_dbm + band3_dbm
    res.loc[index, ['BandI_dBm', 'BandIII_dBm', 'BandI+BandIII_dBm']] = [band1_dbm,band3_dbm,total_dbm]
res['Percentage'] = res['RadioAmount'] / total
res.to_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\6651_config_Sweden_204_Nodes_config_summary.csv', index=False)

In [ ]:
items = self.text_analysis.batch_statistic.table['table'].values
data = pd.DataFrame()
for index, item in enumerate(items):
    print(index)
    try:
        tmp = pd.DataFrame(item)
    except Exception as e:
        print(str(e))
        continue
    data = data.append(tmp, ignore_index=True)
    
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
columns.extend(band1)
columns.extend(band3)
data = data.reindex(columns=columns)
data.to_csv('C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\KaiHong_4480config_origin_data.csv', index=False)

In [ ]:
data = self.text_file.data['dl'].res_key_value
file = pd.DataFrame(pd.Series(data['Band']['value']), columns=['Band'])

file['bxp'] = pd.Series(data['bxp']['value'])
file['rfPort'] = pd.Series(data['rfPort']['value'])
file['rfPort'] = 'Port_' + file['rfPort']
file['carrierType'] = pd.Series(data['carrierType']['value'])
file['Power'] = pd.Series(data['Power']['value'])

result = []
for bxp in set(file['bxp'].values):
    item = {
                'SiteName': self.text_file.file_name.split('.log')[0],
                'Bxp': bxp,
                'Version': self.text_file.data['version'].res_key_value['version']['value'][0]
             }
    tmp = file.loc[(file['bxp'] == bxp), :].reset_index(drop=True)
    for band,bxp,port,carrier,power in tmp.values:
        if band in ['BandI', 'BandIII']:
            cc = band+'_'+port+'_carrier_config'
            pc = band+'_'+port+'_output_power_config'
            dBm = 'output_power_' + band+'_'+port + '_dBm'
            if cc in item:
                item[cc] = carrier + ' + ' + item[cc] if carrier < item[cc] else item[cc] + ' + ' + carrier
                item[pc] = power + ' + ' + item[pc] if carrier < item[cc] else item[pc] + ' + ' + power
                item[dBm] = item[dBm] + round(10**(int(power)/100)/1000, 1)
            else:
                item[cc] = carrier
                item[pc] = power
                item[dBm] = round(10**(int(power)/100)/1000, 1)
    if len(item.keys())> 3:
        result.append(item)

self.result = result

In [ ]:
file_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
output_path = 'D:\\projects\\ericsson_flow\\output'

sheet_names = pd.read_excel(file_path, sheet_name=None).keys()
sheet_names

In [ ]:
pd.read_excel(file_path, sheet_name='Claro_4480_cc_SW', engine='openpyxl')

In [ ]:
import pandas as pd

file_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
output_path = 'D:\\projects\\ericsson_flow\\output'

print('Loading file...')
sheet_names = pd.read_excel(file_path, sheet_name=None).keys()
for sheet_name in sheet_names:
    print(f'Start Handle {sheet_name}...')
    data = pd.read_excel(file_path, sheet_name=sheet_name, engine='openpyxl')
    result = []
    x= list(data['SerialNo'].values)
    sns = sorted(set(x), key=x.index)
    for sn in sns:
        radio = data.loc[(data['SerialNo'] == sn)&(data['MaxTxPower_br'].notna()), :].reset_index(drop=True)
        if len(radio) == 0:
            continue
        item = {'SerialNo': sn, 'SwPack': list(radio['SwPack'].values)[0]}
        for port,band,power in radio[['rfPortId', 'freqBand', 'MaxTxPower_br']].values:
            if band in [1,3]:
                item['Band'+str(int(band))+'_Port'+port+'_MaxTxPower_br'] = power

        band1_power = []
        band3_power = []
        for key in item.keys():
            if 'Band1' in key:
                band1_power.append(item[key])
            if 'Band3' in key:
                band3_power.append(item[key])

        item['Band1_MaxTxPower_br'] = sum(band1_power) if len(band1_power) > 0 else 0
        item['Band3_MaxTxPower_br'] = sum(band3_power) if len(band3_power) > 0 else 0
        item['Band1+Band3_MaxTxPower_br'] = item['Band1_MaxTxPower_br'] + item['Band3_MaxTxPower_br']
        result.append(item)
    data = ''
    del data
    origin_data = pd.DataFrame(result)
    origin_data.to_csv(f'{output_path}\\{sheet_name}_Origin_Data.csv', index=False)

    origin_data = origin_data.fillna('')
    total = len(origin_data)
    summary = origin_data.groupby(list(origin_data.columns[2:])).size().reset_index(name='RadioAmount')
    summary['Percentage'] = summary['RadioAmount'] / total
    summary.to_csv(f'{output_path}\\{sheet_name}_Summary.csv', index=False)

In [ ]:
"""
Script name: my_script.py
Author: DengJun Lei(EEEINLD)
Date created: March 28, 2023
Description: Collect the configuration information for multiple 4480 radios.
Process: 
        1.Extract file returned by 'carrierListHandler print table' command.
        2.Extract (version carrierType rfPort dl/ul Band Power)
        3.Batch extraction and output to a table. 
        4.Export origin data based on a single radio.
        5.Export summary information based on the configuration information of all radios.
"""

###################### User Define ###############################
# Files directory Location
self.dir_path = 'C:\\Users\\MSI-NB\\Downloads\\6651_config_Denmark_68_Nodes\\6651_config_Denmark_68_Nodes'
# Config Location
self.config_path = 'D:\\projects\\ericsson_flow\\configs\\4480_Configure_Statistic_Config.ecfg'
# Output Location
self.output_path = 'D:\\projects\\ericsson_flow\\output'
# Save name
self.save_name = '6651_config_Denmark_68_Nodes'

###################### Batch handle file, according to config ###############################
table = await self.batch_handle(self.dir_path, self.config_path)
table = table.drop(['search_atoms', 'chart_atoms', 'statistic_atoms'], axis=1)

###################### Export origin data ###############################
items = table['table'].values
data = pd.DataFrame()
for index, item in enumerate(items):
    try:
        tmp = pd.DataFrame(item)
    except Exception as e:
        print(str(e))
        continue
    data = data.append(tmp, ignore_index=True)
    
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
columns.extend(band1)
columns.extend(band3)
data = data.reindex(columns=columns)
data.to_csv(f'{self.output_path}//{self.save_name}_origin_data.csv', index=False)
await self.on_console(msg='Export Origin Data Finish!')

###################### Export summary data ###############################
data = data.fillna('')
columns = ['SiteName', 'Bxp', 'Version']
tmpl = sorted(list(set(data.columns).difference(set(columns))))
band1 = []
band3 = []
for c in tmpl:
    if 'BandIII_' in c:
        band3.append(c)
    elif 'BandI_' in c:
        band1.append(c)
        
band1.extend(band3)
columns = band1
total = len(data)
res = data.groupby(columns).size().reset_index(name='RadioAmount')
for index,item in enumerate(json.loads(res.to_json(orient='records'))):
    band1_dbm = []
    band3_dbm = []
    total_dbm = 0
    for key in item.keys():
        if ('output_power_BandIII' in key) & (item[key] != ''):
            band3_dbm.append(item[key])
        elif ('output_power_BandI' in key) & (item[key] != ''):
            band1_dbm.append(item[key])
    band1_dbm = sum(band1_dbm) if len(band1_dbm) > 0 else 0
    band3_dbm = sum(band3_dbm) if len(band3_dbm) > 0 else 0
    total_dbm = band1_dbm + band3_dbm
    res.loc[index, ['BandI_dBm', 'BandIII_dBm', 'BandI+BandIII_dBm']] = [band1_dbm,band3_dbm,total_dbm]
res['Percentage'] = res['RadioAmount'] / total
res.to_csv(f'{self.output_path}//{self.save_name}_summary.csv', index=False)
await self.on_console(msg='Export Summary Finish!')

In [ ]:
"""
Collect the configuration information for 4480 radios xslx.

Author: DengJun Lei(EEEINLD)
CreateTime: 2023.3.20
ChangeTime: 2023.3.30
Version: 1.1
"""

###################### User Define ###############################
# Files directory Location
self.xslx_path = 'D:\\projects\\ericsson_flow\\CC summary V2,0.xlsx'
# Output Location
self.output_path = 'D:\\projects\\ericsson_flow\\output'

print('Loading file...')
sheet_names = pd.read_excel(self.xslx_path, sheet_name=None).keys()
for sheet_name in sheet_names:
    print(f'Start Handle {sheet_name}...')
    data = pd.read_excel(self.xslx_path, sheet_name=sheet_name, engine='openpyxl')
    
    ###################### Export origin data ###############################
    result = []
    x= list(data['SerialNo'].values)
    sns = sorted(set(x), key=x.index)
    for sn in sns:
        radio = data.loc[(data['SerialNo'] == sn)&(data['MaxTxPower_br'].notna()), :].reset_index(drop=True)
        if len(radio) == 0:
            continue
        item = {'SerialNo': sn, 'SwPack': list(radio['SwPack'].values)[0]}
        for port,band,power in radio[['rfPortId', 'freqBand', 'MaxTxPower_br']].values:
            if band in [1,3]:
                item['Band'+str(int(band))+'_Port'+port+'_MaxTxPower_br'] = power

        band1_power = []
        band3_power = []
        for key in item.keys():
            if 'Band1' in key:
                band1_power.append(item[key])
            if 'Band3' in key:
                band3_power.append(item[key])

        item['Band1_MaxTxPower_br'] = sum(band1_power) if len(band1_power) > 0 else 0
        item['Band3_MaxTxPower_br'] = sum(band3_power) if len(band3_power) > 0 else 0
        item['Band1+Band3_MaxTxPower_br'] = item['Band1_MaxTxPower_br'] + item['Band3_MaxTxPower_br']
        result.append(item)
    data = ''
    del data
    origin_data = pd.DataFrame(result)
    origin_data.to_csv(f'{self.output_path}\\{sheet_name}_Origin_Data.csv', index=False)
    
    ###################### Export summary data ###############################
    origin_data = origin_data.fillna('')
    total = len(origin_data)
    summary = origin_data.groupby(list(origin_data.columns[2:])).size().reset_index(name='RadioAmount')
    summary['Percentage'] = summary['RadioAmount'] / total
    summary.to_csv(f'{self.output_path}\\{sheet_name}_Summary.csv', index=False)

In [ ]:
"""
Script name: DOT4455_Abnormal_Analyze.escp
Author: DengJun Lei(EEEINLD)
Date created: April 11, 2023
Description: Batch analyze DOT4455 abnormal keywords and group them based on RU, RdPort, and Abnormal.
"""

###################### User Define ###############################
group = ['Root','RU','RP','Abnormal']
# Files directory Location
self.dir_path = 'D:/projects/ericsson_flow/new_files/low_tx_power_log0331'
self.exps = [
                {'name': 'DPD errorCode 0x340a', 'exp': '{RU}: OK {} {timestampd} {timestampt} {}: DPD errorCode 0x340a{}'},
                {'name': 'Low tx Power', 'exp': '{RU}: OK {} {timestampd} {timestampt} {}: Low tx Power{}'}
            ]
self.end_marker = '{}Done RdId:{RI}, RdPort:{RP}{}'
origin_data_output = 'D:/projects/ericsson_flow/new_files/OriginData.csv'
root_name = 'DOT4455'
pixel_width = 5000

###################### Execution area ###############################
await self.on_console(msg='Script running...')
# extract keywords and generate origin data
result = pd.DataFrame()
for file_name in iterate_files_in_directory(self.dir_path):
    path = f'{self.dir_path}\\{file_name}'
    with open(path, 'r') as f:
        lines = f.readlines()
    
    rd_port = []
    for line in lines:
        for exp in self.exps:
            r = parse(exp['exp'], line)
            if r is not None:
                rd_port.append({'IP': file_name, 'RU': r['RU'], 'Abnormal': exp['name'], 'timestamp': convert_datetime_timestamp(r['timestampd'] + ' ' + r['timestampt'])})
        if parse(self.end_marker, line) is not None:
            tmp = pd.DataFrame(rd_port)
            tmp['RP'] = parse(self.end_marker, line)['RP']
            result = result.append(tmp).reset_index(drop=True)
            rd_port = []
    await self.on_console(msg=f'Finish {file_name}.')
result['Root'] = root_name
result['RP'] = result['RP'].astype(str)
result = result.loc[:, ['Root','IP','RU','RP','Abnormal','timestamp']]
result.to_csv(origin_data_output, index=False)

# result = pd.read_csv('D:\\projects\\ericsson_flow\\new_files\\OriginData.csv')
# result['RP'] = result['RP'].astype(str)

# organize the required data.
items = []
grouped = result.groupby(group)\
    .apply(lambda x: x[['IP', 'timestamp']].assign(api='that.textAnalysisView.fileContainerView.controlNewFile(["'+self.dir_path+'/'+x['IP']+'"])').to_dict('records'))\
    .reset_index(name='data')
group.append('data')
for p in grouped[group].values:
    groups = p[0: -1]
    data = p[-1]
    items.append({'path': list(groups), 'graph_type': 'ScatterPlot', 'start_x': min(pd.DataFrame(data)['timestamp'].values), 'end_x': max(pd.DataFrame(data)['timestamp'].values), 'elements': data})

start_x = []
end_x = []
for item in items:
    start_x.append(item['start_x'])
    end_x.append(item['end_x'])

graphs = []
# define xaxis
graphs.append({
    'type': 'XAxis',
    'width': pixel_width,
    'lower_bound': min(start_x),
    'upper_bound': max(end_x),
    'tick_format_func': 'formatTimestamp'
})

# define IndentedTree
global_inter = linear_scale([min(start_x), max(end_x)], [0, pixel_width])
tree = IndentedTree('', 0, pixel_width, pixel_width, items, global_inter)
graphs.append({'id': tree.id, 'type': tree.type, 'elements': tree.elements})

nodes = tree.get_all_nodes_to_list()
for node in nodes:
    if node['elements'] is not None:
        sp = ScatterPlot(node['id'],node['sx'],node['ex'],node['width'],node['elements'],global_inter,2,'timestamp')
        graphs.append(node)

graphs.append({
    'type': 'Brush',
    'width': pixel_width,
    'height': tree.height
})
await self.on_draw('', graphs)

In [29]:
"""
Script name: AIR3268B42_Abnormal_Analyze.escp
Author: DengJun Lei(EEEINLD)
Date created: April 12, 2023
Description: Batch analyze AIR3268B42 abnormal keywords and group them based on file_name,product_name,assigned_name,cmd,keywords, and Abnormal.
"""

# from utils import *
# from graph import *
# import random

###################### User Define ###############################
group = ['file_name','product_name','assigned_name','cmd','keywords']
# Files directory Location
dir_path = 'D:/projects/ericsson_flow/new_files/AIR3268B42_log'

exps = {
    'trx status': {'extract_exps': {
                    'timestamp':{'exp':'{}Date: {timestamp},{}', 'cond': "row['timestamp_abnormal'] = False"},
                    'txPma':{'exp':'{}txPma{}: {txPma},{}', 'cond': "row['txPma_abnormal'] = False"},
                    'txDpdPma':{'exp':'{}txDpdPma{}: {txDpdPma},{}', 'cond': "row['txDpdPma_abnormal'] = True if float(row['txPma']['value']) - float(row['txDpdPma']['value']) > 4 else False"},
                    'txPmb':{'exp':'{}txPmb{}: {txPmb},{}', 'cond': "row['txPmb_abnormal'] = True if float(row['txPma']['value']) - float(row['txPmb']['value']) != 0 else False"},
                    'txTorPmb':{'exp':'{}txTorPmb{}: {txTorPmb},{}', 'cond': "row['txTorPmb_abnormal'] = True if float(row['txPma']['value']) - float(row['txTorPmb']['value']) > 0.1 else False"},
                    'dpd':{'exp':'{}dpd {}: {dpd},{}', 'cond': "row['dpd_abnormal'] = True if row['dpd']['value'] in ['off'] else False"},
                    'dpdStateMachine':{'exp':'{}dpdStateMachine{}: {dpdStateMachine},{}', 'cond': "row['dpdStateMachine_abnormal'] = True if row['dpdStateMachine']['value'] in ['OFF'] else False"},
                    'gainStateMachine':{'exp':'{}gainStateMachine{}: {gainStateMachine},{}', 'cond': "row['gainStateMachine_abnormal'] = True if row['gainStateMachine']['value'] in ['CtrlGainStateIdle:started', 'ns:stopped'] else False"},
                    'linearizationStateMachine':{'exp':'{}linearizationStateMachine{}: {linearizationStateMachine},{}', 'cond': "row['linearizationStateMachine_abnormal'] = True if row['linearizationStateMachine']['value'] in ['ns:stopped'] else False"},
                }, 'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    'elog read': {'extract_exps': {
                    'LinFault':{'exp':'{}[{timestamp}]{}Lin. fault port{LinFault}', 'cond': "row['LinFault_abnormal'] = True if row['LinFault'] is not None else False"},
                    'PowerLost':{'exp':'{}[{timestamp}]{}POWER LOST{PowerLost}', 'cond': "row['PowerLost_abnormal'] = True if row['PowerLost'] is not None else False"},
                    'PsuEnter':{'exp':'{}[{timestamp}]{}PSU enters{PsuEnter}', 'cond': "row['PsuEnter_abnormal'] = True if row['PsuEnter'] is not None else False"},
                    'JesdLinkFailure':{'exp':'{}[{timestamp}]{}JESD LINK FAILURE{JesdLinkFailure}', 'cond': "row['JesdLinkFailure_abnormal'] = True if row['JesdLinkFailure'] is not None else False"},
                    'DpdController':{'exp':'{}[{timestamp}]{}#### DPDCONTROLLER{DpdController}', 'cond': "row['DpdController_abnormal'] = True if row['DpdController'] is not None else False"},
                }, 'time_type': 'embedded', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} elog read{}', 'end_exp': '{}End of log{}'},
}

map_table_start_mark = ['FRU','LNH','BOARD','ST','FAULT','OPER','MAINT']
map_table_end_mark = '---------------------------------------------------------------------'
map_table_exp = '{} ;{assigned_name} ;{product_name} {}'
origin_data_output = 'D:/projects/ericsson_flow/new_files/AIR3268OriginData.csv'
pixel_width = 2000

###################### Execution area ###############################
# await self.on_console(msg='Script running...')
# extract keywords and generate origin data
def abnormal_condition(row, code):
    exec(code)
    return row
    
result = pd.DataFrame()
for file_index, file_name in enumerate(iterate_files_in_directory(dir_path)):
    path = f'{dir_path}\\{file_name}'
    with open(path, 'r') as f:
        lines = f.readlines()
    
    # find map table
    flag = False
    table = {}
    for line in lines:
        if list(set([True if word in line else False  for word in map_table_start_mark]))[0] == True:
            flag = True
        if flag:
            r = parse(map_table_exp, line)
            if r is not None:
                table[r.named['assigned_name']] = r.named['product_name']
        if (map_table_end_mark in line) & flag:
            break

    # init 
    search_contents = {}
    for key in exps.keys():
        search_contents[key] = []
    
    # find search content
    for key in exps.keys():
        flag = False
        content = {'lines':[]}
        for index, line in enumerate(lines):
            if flag:
                r = parse(exps[key]['end_exp'], line)
                if r is not None:
                    flag = False
                    search_contents[key].append(content)
                    content = {'lines':[]}
                else:
                    content['lines'].append({'global_index':index, 'text':line})
                    continue
            r = parse(exps[key]['start_exp'], line)
            if r is not None:
                flag = True
                content['assigned_name'] = r['assigned_name']
                content['lines'].append({'global_index':index, 'text':line})
    
    # extract key value
    res = []
    for key in search_contents.keys():
        for cmd_batch, content in enumerate(search_contents[key]):
            tmp = {}
            for extract_key in exps[key]['extract_exps'].keys():
                if extract_key != 'timestamp':
                    tmp[extract_key] = []
            
            if exps[key]['time_type'] == 'embedded':
                for line in content['lines']:
                    for extract_key in exps[key]['extract_exps'].keys():
                        r = parse(exps[key]['extract_exps'][extract_key]['exp'], line['text'])
                        if r is not None:
                            item = {
                                    'file_name':file_name, 'product_name':table[content['assigned_name']], 'assigned_name':content['assigned_name'], 
                                    'cmd':key, 'keywords':extract_key, 'global_index': line['global_index'], 'value':'',
                                    'cmd_batch': cmd_batch, 'timestamp': convert_datetime_timestamp(r.named['timestamp']), 'x': random.randint(0, 100), 'y':random.randint(0, 100)
                                    }
                            tmp[extract_key].append(item)
            elif exps[key]['time_type'] == 'batch':
                timestamp = ''
                for line in content['lines']:
                    for extract_key in exps[key]['extract_exps'].keys():
                        r = parse(exps[key]['extract_exps'][extract_key]['exp'], line['text'])
                        if r is not None:
                            if 'timestamp' in r.named:
                                timestamp = convert_datetime_timestamp(r.named['timestamp'])
                                break
                            item = {
                                    'file_name':file_name, 'product_name':table[content['assigned_name']], 'assigned_name':content['assigned_name'], 
                                    'cmd':key, 'keywords':extract_key, 'global_index': line['global_index'], 'value':r.named[extract_key],
                                    'cmd_batch': cmd_batch, 'timestamp': timestamp, 'x': random.randint(0, 100), 'y':random.randint(0, 100)
                                    }
                            tmp[extract_key].append(item)
                
            for keyword in tmp.keys():

                max_len = max([len(v) for v in tmp.values()])
                for k, v in tmp.items():
                    if len(v) < max_len:
                        v.extend([None] * (max_len - len(v)))
        
                an = keyword+'_abnormal'
                ptmp = pd.DataFrame(tmp)
                ptmp = ptmp.apply(lambda x: abnormal_condition(x, exps[key]['extract_exps'][keyword]['cond']), axis=1)
                abnormal = ptmp.loc[(ptmp[an] == True), keyword].values
                res.append({'file_name': file_name, 'product_name': table[content['assigned_name']],'assigned_name': content['assigned_name'], 'cmd': key, 'keywords': keyword, 'abnormal':abnormal})
    data = pd.DataFrame(res).groupby(group)['abnormal'].apply(lambda x: [i for j in x for i in j]).reset_index(name='merged_abnormal')
    await self.on_console(msg=f'Finish {file_index} {file_name}.')
result = result.append(data).reset_index(drop=True)
result.to_csv(origin_data_output, index=False)

# draw picture
items = []
group.append('merged_abnormal')
for p in result[group].values:
    groups = [i.replace('.','_') for i in p[0: -1]]
    data = p[-1]
    ts = []
    for dot in data:
        ts.append(dot['timestamp'])
    items.append({'path': list(groups), 'graph_type': 'ScatterPlot', 'start_x': min(ts) if len(ts) > 0 else 0, 'end_x': max(ts) if len(ts) > 0 else 0, 'elements': data})

start_x = []
end_x = []
for item in items:
    if (item['start_x'] !=0) & (item['end_x'] !=0):
        start_x.append(item['start_x'])
        end_x.append(item['end_x'])

graphs = []
# define xaxis
graphs.append({
    'type': 'XAxis',
    'width': pixel_width,
    'lower_bound': min(start_x),
    'upper_bound': max(end_x),
    'tick_format_func': 'formatTimestamp'
})
# define IndentedTree
global_inter = linear_scale([min(start_x), max(end_x)], [0, pixel_width])
tree = IndentedTree('', 0, pixel_width, pixel_width, items, global_inter)
graphs.append({'id': tree.id, 'type': tree.type, 'elements': tree.elements})

nodes = tree.get_all_nodes_to_list()
for node in nodes:
    if node['elements'] is not None:
        sp = ScatterPlot(node['id'],node['sx'],node['ex'],node['width'],node['elements'],global_inter,2,'x')
        for dot in sp.elements:
            dot['api'] = f"""
                that.textAnalysisView.fileContainerView.controlNewFile(["{dir_path}/{dot['file_name']}"])
                setTimeout(function() {{
                   that.textAnalysisView.fileContainerView.textFileViews[that.textAnalysisView.fileContainerView.activeTextFileView].textFileOriginalView.controlJump(d)
                }}, 500)
            """
        graphs.append(sp.get_vars())

graphs.append({
    'type': 'Brush',
    'width': pixel_width,
    'height': tree.height
})
await self.on_draw('', json.dumps(graphs))

C:\Users\LDJ\AppData\Local\Temp\ipykernel_18468\1494805047.py:145: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(data).reset_index(drop=True)


In [17]:
ptmp = pd.DataFrame(tmp)
ptmp.loc[30,'txDpdPma']

{'file_name': 'DL-NL1-1-202303301315.log',
 'product_name': 'AIR3268B42',
 'assigned_name': 'fru_2053',
 'cmd': 'trx status',
 'keywords': 'txDpdPma',
 'global_index': 15423,
 'value': -33.29,
 'cmd_batch': 2,
 'timestamp': '2023-03-30 07:47:15',
 'x': 9,
 'y': 52}

In [20]:
for a, b in zip(tmp['txPma'], tmp['txDpdPma']):
    print(a['global_index'], b['global_index'])

10895 10896
11041 11042
11187 11188
11333 11334
11480 11481
11629 11630
11775 11776
11921 11922
12066 12067
12212 12213
12358 12359
12504 12505
12650 12797
12796 12943
12942 13088
13087 13234
13233 13380
13379 13526
13525 13672
13671 13818
13817 13964
13963 14110
14109 14256
14255 14402
14401 14548
14547 14694
14693 14840
14839 14986
14985 15131
15130 15277
15276 15423


TypeError: 'NoneType' object is not subscriptable